# DSO 499 Milestone Report

### Team: Earth

### Team Members:
1. Joao Ama - wrote demand, customer_location, stock_creator functions and helped with the weeksimulator and main loop logic
2. Kim Nguyen - wrote profit function, helped with weeksimulator and main, editing/cleaning code
3. Teeny Chen - wrote the distance function using functions written by other members and implemented it into the weeksimulator and main functions

### Overall Structure:
Part 1: defining variables and importing data

Part 2: Defined functions relating to customer

Part 3: Defined functions to calculate profit

Part 4: Defined function to simulate weekly profit using functions above

Part 5: main functions simulates profit for different stock combinations for 2 year period, using functions defined above


### Program:

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from geopy import distance

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

ModuleNotFoundError: No module named 'geopy'

## Milestone

In [ ]:
# Enter Data
warehouses = ['Joymode Downtown LA', 'Joymode South Pasadena']
regions = ['Agoura Hills-Malibu', 'Compton', 'Downey-Norwalk', 'East San Gabriel Valley', 'Inglewood', 'Long Beach-Lakewood', 'Los Angeles', 'Palos Verdes', 'Pasadena', 'San Fernando Valley', 'Santa Monica', 'South Bay Cities', 'South Gate-East Los Angeles', 'Southwest San Gabriel Valley', 'Torrance', 'Upper San Gabriel Valley', 'Whittier']
region_pop = [63824, 342576, 414144, 935875, 369694, 569100, 2504251, 91365, 268040, 1771545, 89736, 138043, 501237, 318982, 145438, 323899, 316911]

# distance[r][w] is the great-circle distance (in miles) between the region indexed r and the warehouse indexed w
distances = [[28.01340494, 34.81062341],
           [11.8651692 , 18.13995573],
           [13.1113517 , 15.24304705],
           [23.40306115, 17.2256941 ],
           [ 9.03251773, 17.23831517],
           [18.58433256, 23.12760706],
           [ 6.17370848, 14.7138748 ],
           [19.70968228, 27.44264297],
           [11.2286649 ,  3.515125  ],
           [16.84645184, 17.66079915],
           [13.50731777, 21.30107062],
           [14.23180632, 22.61968895],
           [ 5.88718363,  9.46398148],
           [ 8.84474521,  4.77595723],
           [15.4041491 , 23.23103475],
           [14.62970169,  7.12125806],
           [14.61958346, 13.22239959]]



customer_base = 3000
rent_interval = 12
stock1list = []
stock2list = []

In [ ]:
# some data for calculating profit
rentPrice = 65  # price to rent bundle for weekend, includes delivery both ways
retailValue = 439  # ALSO PRICE JOYMODE PAYS FOR CAPITAL INVESTMENT
capitalRate = 439  # price joymode pays to get bundle
flashPrice = 150  # EVERY 2 YEARS, bundles are sold at flash sales for this price
deliveryRate = 0.72  # 0.72 PER MILE!!!! including fuel and labor
cleaningRate = 5  # 5 dollars PER BUNDLE

In [ ]:
def demand(base, interval):
   '''
   This generates a random number of customers that want to rent on a given week.
   Input: Size of customer base, the interval at which customers rent
   Return: The expected demand based on exponential distribution
   '''
   a = 1 / interval
   totaldemand = 0
   for b in range(base): #tried to do np.random.exponential(250),but there was a huge variation. This code works
       totaldemand += np.random.exponential(a)
   return totaldemand

In [ ]:
def customer_location(regions, region_pop):
   '''
   This says where a specific customer is located, randomly generated based on population distribution
   Input: The list of regions and the list of populations in each of those regions
   Return: Which region was selected
   '''
   # this says where a specific customer is located,
   #  randomly generated based on population distribution
   total_population = 0
   prob = []
   for i in region_pop:
       total_population += i
   for i in region_pop:
       p = i / total_population
       prob.append(p)
   customer_region = np.random.choice(regions, p=prob)
   return customer_region

In [ ]:
def distance(customer_region):
   '''
   Return the warehouse that the customer is closest to and the distance between them in miles.
   Input:
       customer_region - the region the specific customer is located in
   Return:
       closestDistance - the distance between the closest warehouse and the region the customer is in (in miles)
       dist.index(closestDistance) - the index of the closest warehouse to the customer in the warehouse list (0 for DTLA, 1 for South Pasadena)
   '''

   # Determine index of the region customer is in in the regions list
   i = regions.index(customer_region)

   # Find the closest warehouse
   dist = []   # store distances between the customer_region and warehouses
   for j in range(len(warehouses)):
       dist.append(distances[i][j])

   closestDistance = min(dist)

   return closestDistance, dist.index(closestDistance)

In [ ]:
def calculateProfit(distanceofCustomer):
   '''
   Calculates profit FOR ONE CUSTOMER.
   input: the distance of the customer to warehouse item is being shipped from
   return: the profit made from that one customer. Profit = revenue - cleaning costs - delivery costs
   '''
   deliveryCost = deliveryRate * distanceofCustomer
   profit = rentPrice - cleaningRate - deliveryCost
   return profit

In [ ]:
def weeksimulator(stock1, stock2):
   '''
   This function simulates a week, runs through every customer that wanted to rent that week, and calculates
   the total week's profit.
   inputs:
       stock1 = amount of stock from warehouse 1
       stock2 = amount of stock from warehouse 2
   returns: total week's profit
   '''
  
   # STEP 1: DETERMINE DEMAND FOR THE WEEK
   customers = int(demand(customer_base, rent_interval))

   # STEP 2: CALCULATE THE PROFIT EARNED THAT WEEK
   profit = 0

   # for each customer
   for customer in range(customers):

       # determine which region they are from
       region = customer_location(regions, region_pop)
       # determine which warehouse is closest to customer and how far they are from the warehouse
       distanceOfCustomer, warehouse = distance(region)  # distanceOfCustomer is the number of miles customer is from warehouse
       # if first warehouse customer is closer to first warehouse
       if warehouse == 0:  
           # check if the warehouse has stock
           if stock1 > 0:
               # simulate transaction
               # calculate profit received from transaction and updates stock
               profit += calculateProfit(distanceOfCustomer)  # this uses the actual distance to calculate the profit
               stock1 -= 1  # updates stock

           # if the closer warehouse is out of stock
           else:  # stock1<0
               # check if other warehouse has stock
               if stock2 > 0:
                   # update the distance to be distance between customer and second warehouse
                   distanceOfCustomer = distances[warehouse][1]
                   profit += calculateProfit(distanceOfCustomer)
                   stock2 -= 1

               # if the other warehouse doesn't have it, they lose the customer
               # nothing happens to their profit
               else:
                   pass

       if warehouse == 1:
           if stock2 > 0:
               profit += calculateProfit(distanceOfCustomer)
               stock2 -= 1
           else:  # stock2 < 0
               if stock1 > 0:
                   # update the distance to be distance between customer and first warehouse
                   distanceOfCustomer = distances[warehouse][0]
                   profit += calculateProfit(distanceOfCustomer)
                   stock1 -= 1
               else:
                   pass

   return profit

In [ ]:
def stockcreator():
   '''
   This function creates the list of different stock options from 0 to 500 increasing by 50
   Returns: Two lists, stock1 for the Downtown warehouse and stock2 for the Pasadena warehouse
   '''
   q = 0
   while q < 500:
       stock1list.append(q)
       stock2list.append(q)
       q += 50
   return stock1list, stock2list



In [ ]:
def main():
   '''
   Simulates the profit for 2 years for many samples
   Calculates the average profit for the simulations for each stock combinations
   It prints the average profit and the corresponding stocks
   It prints out the largest average profit and the stock combinations that led to the largest profit
   '''

   largest = 0
   simulations = 5
   year = 52
   stockcreator()
   maxstock1 = 0
   maxstock2 = 0


   print("Average Profit, Stock 1, Stock 2")
   for stock1 in stock1list:  # this runs through all the combination of stock at warehouse1
       for stock2 in stock2list:  # this runs through all the combinations of stock at warehouse2

           yearProfitofAllSimulations = 0

           for simulation in range(simulations):  # this runs x number of simulations

               yearProfit = 0  # 2 year profit of 1 simulation

               for week in range(2 * year):  # this simulates the profit for two full years
                   yearProfit += weeksimulator(stock1, stock2)

               yearProfit -= (stock1 + stock2) * (439 - 150)  # this subtracts the cost we spent buying the machines

               yearProfitofAllSimulations += yearProfit

           AvgProfit = yearProfitofAllSimulations / simulations

           # keeping track of largest average profit and corresponding stock values
           if AvgProfit > largest:
               largest = AvgProfit
               maxstock1 = stock1
               maxstock2 = stock2

           print(round(AvgProfit), stock1, stock2)

   print("Largest 2 year Profit: ", round(largest, 2))  # this is the average profit for a 2 year run for the optimal inventory levels
   print("Downtown Stock: ", maxstock1, "Pasadena Stock:", maxstock2)  # these are the optimal inventory levels

In [ ]:
main()

## 0. Load Data

In [ ]:
warehouse_data = pd.read_csv('/kaggle/input/joymode_warehouses.csv')
cust_regions = pd.read_csv('/kaggle/input/customer_regions.csv')

In [ ]:
warehouse_data = warehouse_data.set_index('Name')

In [ ]:
warehouse_data.head()

In [ ]:
cust_regions.head()

In [ ]:
print(distance.distance((34.040461,-118.266415), (33.869642, -118.245185)).miles)

## 1. Warehouse class

In [32]:
# Warehouse class object
class Warehouse:
    
    def __init__(self,name):
        '''Constructor method
        :param data: warehouse pandas DataFrame
        :param name: Name of the warehouse to look up
        '''
        self.name = name
        self.inventory = [50,50,50,50,50]
        self.deliveryList = {"Santa Monica":0,"USC":0,"UCLA":0}
        #self.address = data.loc[name,:].Address
        #self.long = data.loc[name,:].Longitude
        #self.lat = data.loc[name,:].Latitude
        
    def checkInventory(self,bundle):
        return self.inventory[bundle] != 0 
            # returns True if there is inventory, returns false if out of inventory
                
        

## 2. Customer class

In [2]:
import random
import numpy as np

bundles = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] #these are the 12 bundles that Joymode sells

class Customer:
    def __init__(self):
        self.region = ""
        self.desire = []
        self.satisfaction = 1

    def generateLocation(self, region_pop, regions):
        '''
        This says where a specific customer is located, randomly generated based on population distribution
        Input: The list of regions and the list of populations in each of those regions
        Return: Which region was selected
        '''
        # this says where a specific customer is located,
        #  randomly generated based on population distribution
        total_population = 0
        prob = []
        for i in region_pop:
            total_population += i
        for i in region_pop:
            p = i / total_population
            prob.append(p)
        self.region = np.random.choice(regions, p=prob)
        return self.region

    def generateDesire(self, bundles):
        '''
        This generates a list of a customer top three bundles based on a equal probability of desire for each bundle
        Input: The list of bundles
        Return: The generated list of desires
        '''
        desire = np.random.choice(bundles, 3, replace=False)
        self.desire = desire
        return self.desire

    def leave(self):
        '''
        This deletes the object if their satisfaction drops below a certain level
        Returns: True which means that the customer did leave
        '''
        if self.satisfaction < .6:
            del self
            return True
         #not sure if this will work

    def lowerSatisfaction(self, satisfaction):
        self.satisfaction = satisfaction

    def getSatisfaction(self):
        return self.__satisfaction

## 3. Joymode class

In [21]:
import csv
#import Customer from Customer

class Joymode:
    def __init__(self):
        self.warehouses = {}
        self.customers = []
        self.profit = 0
        self.customerslost = 0
        self.customerbase = 10 #3000
        self.customersatisfaction = 1

    def createCustomers(self,household,regions): # might have to be edited 
        '''
        This program will create 3000 individual customers, generate their locations, and add them to our warehouse list
        '''
        for i in range(self.customerbase):
            a = Customer()
            a.generateLocation(household, regions)
            self.customers.append(a)

    def createWarehouses(self, warehouselist):
        '''
        This function will populate the warehouse class with the warehouses we were given and add them to our warehouse list
        '''
        for name in warehouselist: # i IS NAME 
            self.warehouses[name]=Warehouse(name)
            
    def getWeekDemand(avgCustomer):
       '''
       This picks out randomn number of customers that want to rent on a given week, using exponential distribution. 
       Calculates the number of customers that will order that week, ie. 250
       Randomly selects 250 customers from the all the customers at Joymode. 
       Input: average number of customers that order per week
       Return: list containing customer objects, representing customers that will order that week 
       '''
        # generate number of customers 
        numCustomers = 0
        time = 0
        a = 1/avgCustomer
        while time < 1: 
            time += np.random.exponential(a)
            numCustomers += 1
        numCustomers -= 1 # to subtract the last person, because their time is over 250
        numCustomers
        import random
        # select customers from customer base 
        customerList = random.sample(self.customers, numCustomers)
        return customerList
        
    def getSatisfaction(self):
        '''
        This function gets the satisfaction of every customer and creates an avg customer satisfaction
        Returns: Average Customer Satisfaction
        '''
        for customer in self.customers:
            self.customersatisfaction += customer.getSatiscation()
        self.customersatisfaction /= self.customerbase
        return self.customersatisfaction
    
    def findClosetTwoWarehouse(self,customerLocation):
        '''
        Input: customer's location 
        Returns: list of the INDEX TWO CLOSEST warehouse to the customer
        '''
        # df.loc[customerLocation].sort_values()[0:2] this returns the DISTANCE!!! NOT NAME
        houseIndex = df2.loc[customerLocation].argsort()[0:2] # THIS RETURNS THE INDEX
        warehouseName = df2.columns[houseindex].tolist() # THIS RETURNS THE NAME
        return warehouseName
    
    def checkWarehouses(customerChoices,warehouseList):
    '''
    Input: list containing customer's top 3 choices, list containing 2 closest warehouse to customer
    Returns: the customer's choice if it is in stock and NAME of closest warehouse that has item
    '''
    for choice in customerChoices:
        # check warehouse list
        for name in warehouseList:
            # if choice exists in warehouse then return it
            if Joymode2.warehouses[name].checkInventory(choice) == True:
                return choice, name
    return "noStock" # return if none of customer's choices are in stock at 2 closest warehouse
    
    def simulateWeek(self):
        '''
        This program runs the weekly simulation
        '''
        for customer in self.customers:
            desire = customer.generateDesire(bundles)
            #use a program that takes desire as input and then outputs which warehouse is delivering and which product
            #subtract product from warehouse
            #if choice == 2:
                #customer.lowerSatiscation(.8)
            #elif choice == 3:
                #customer.lowerSatisfaction(.7)
            #elif

    def export(self):
        '''
        This function creates a csv file containing the program results
        '''
        with open('joymode_results.csv', mode='w') as joymode_results: ##im not sure how to include inventory levels for each area, is each one a column?
            joymode_results = csv.writer(joymode_results, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            joymode_results.writerow(['Profit', 'Number of Customers', 'Customers Lost', 'Customer Satisfaction'])
            joymode_resultsmode_results.writerow([self.profit, self.customerbase, self.customerslost, self.customersatisfaction])
            

In [15]:
import pandas as pd
df2 = pd.DataFrame({'Regions': ["Santa Monica", "USC", "UCLA"],
                   'DTLA': [2012, 2014, 2013],
                   'Pasadena':[55, 40, 84],'Culver City':[25, 50, 67]})
df2.set_index("Regions",inplace=True)
#df2.loc['USC']
warehouse=df2.loc["USC"].sort_index(ascending=False)
warehouse

Pasadena         40
DTLA           2014
Culver City      50
Name: USC, dtype: int64

### demand generator

In [16]:
houseindex = df2.loc["USC"].argsort()[0:2]
df2.columns[houseindex].tolist()

['Pasadena', 'Culver City']

In [17]:
df2

,DTLA,Pasadena,Culver City
Regions,,,
Santa Monica,2012,55,25
USC,2014,40,50
UCLA,2013,84,67


In [18]:
mydict = {}
mydict["USC"] = 0 
mydict["USC"] += 1
mydict

{'USC': 1}

In [19]:
import numpy as np
def demand(avgCustomer):
   '''
   This generates a random number of customers that want to rent on a given week, using exponential distribution. 
   Input: average number of customers that order per week
   Return: the expected number of customer orders for one week
   '''
   numCustomers = 0 
   time = 0
   a = 1/avgCustomer
   while time < 1: 
    time += np.random.exponential(a)
    numCustomers += 1
   numCustomers -= 1 # to subtract the last person, because their time is over 250
   return numCustomers
# NEXT: write function that randomly chooses this number of people from Joymode's 3000 customer objects



## Def bigProject(self, customerlocation, customerdesire, distance, inventory)
-This function takes in the customer location, the customer’s desire, the distance from the customer to all the different warehouses, and the inventory of all the warehouses

-It then checks the customer’s desire to see if it is available at the nearest warehouse, if not, it checks the second nearest warehouse

-If both dont have it, it then checks the customers second choice in the same manner

-Then the third choice, if none of these choices are fulfilled, the customer gets nothing

-Returns: which choice the customer got fulfilled (a single number) , which warehouse is fulfilling it, choice index 


# *** SAMPLE: iteration for one customer *** 

In [40]:
# create customer
customer1 = Customer()
customer1.region = "USC"

# generate customer's desire
desire = customer1.generateDesire([0,1,2,3,4])
desire # [1,4,5]
Joymode2 = Joymode() 

# create warehouse objects
warehousestoCreate = ["Pasadena","Culver City"]
Joymode2.createWarehouses(warehousestoCreate)

In [41]:
# find 2 closests warehouses to the customer
warehouseList = Joymode2.findClosetTwoWarehouse(customer1.region) # LIST of 2 NAME of the closet warehouse
warehouseList

['Pasadena', 'Culver City']

In [44]:
# check if warehouse 
def checkWarehouses(customerChoices,warehouseList):
    '''
    Input: list containing customer's top 3 choices, list containing 2 closest warehouse to customer
    Returns: the customer's choice if it is in stock and NAME of closest warehouse that has item
    '''
    for choice in customerChoices:
        # check warehouse list
        for name in warehouseList:
            # if choice exists in warehouse then return it
            if Joymode2.warehouses[name].checkInventory(choice) == True:
                return choice, name
    return "noStock" # return if none of customer's choices are in stock at 2 closest warehouse
            
            
        
test = checkWarehouses()
test

2

In [43]:
Joymode2.warehouses["Pasadena"].checkInventory(1)


True

In [38]:
la = Warehouse("la")
la.checkInventory(1)

True

In [ ]:
warehouses = sample.findClosetTwoWarehouse(customer1.region) # LIST of 2 NAME of the closet warehouse
for choice in desire:
    # check warehouse list
    for warehouse in warehouses:
        # if choice exists in warehouse then return it and break 
        if Joymode2.warehouse[warehouses[i]].checkInventory(choice) == True:
            return choice
        else 
            continue
        # else: then continue 